In [ ]:
import sys
sys.path.append("..")
import numpy as np
import scipy.stats
from stats import KDE, GaussianMixture
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# With dependent variable

In [ ]:
n1 = 500
n2 = 500
ythreshold = 3
gm = GaussianMixture([[2, 2], [-2, -2]], [[[1, .5], [.5, 1]], np.eye(2)])
(xpdf, ypdf), zpdf = gm.pdf(minx=[-5, -5], maxx=[10, 10])
xpdf = xpdf[0]
pdfreal = np.sum(zpdf, axis=0) * np.mean(np.diff(ypdf[:, 0]))

(xpdf1, ypdf1), zpdf1 = gm.pdf(minx=[-5, -5], maxx=[10, ythreshold])
(xpdf2, ypdf2), zpdf2 = gm.pdf(minx=[-5, ythreshold], maxx=[10, 10])
intpysmaller = np.trapz(zpdf1, ypdf1[:, 0], axis=0)
intpybigger = np.trapz(zpdf2, ypdf2[:, 0], axis=0)
pybigger = intpybigger / (intpysmaller + intpybigger)
py0 = np.trapz(intpybigger, xpdf)  # This is the probability that normal data is also edge data
plt.plot(xpdf, pybigger)
print(f"Probability of edge case data: {py0:.3f}")

In [ ]:
def try_only_normal(seed):  # Only using normal data
    np.random.seed(seed)
    data = gm.generate_samples(n1)[:, 0]
    k = KDE(data, scaling=True)
    k.compute_bandwidth()
    pdfest = k.score_samples(xpdf)
    return pdfest

In [ ]:
def try_method(seed):
    np.random.seed(seed)
    
    data1 = gm.generate_samples(n1)
    data2 = gm.generate_samples(int(n2/py0*2))  # times 2, so we have enough
    data2 = data2[data2[:, 1] > ythreshold, 0]
    if len(data2) > n2:  # It almost certainty is
        data2 = data2[:n2]
    normal_data = data1[data1[:, 1] <= ythreshold, 0]
    edge_data = np.concatenate((data2, data1[data1[:, 1] > ythreshold, 0]))

    kx1 = KDE(normal_data, scaling=True)
    kx1.compute_bandwidth()
    pdfest1 = kx1.score_samples(xpdf)

    kx2 = KDE(edge_data, scaling=True)
    kx2.compute_bandwidth()
    pdfest2 = kx2.score_samples(xpdf)
    
    return pdfest1, pdfest2

In [ ]:
pdf1, pdf2 = try_method(0)
plt.plot(xpdf, pdfreal, label="Real")
plt.plot(xpdf, pdf1, label="Est1")
plt.plot(xpdf, pdf2, label="Est2")
plt.plot(xpdf, pdf1*(1-py0)+pdf2*py0, label="Combined")
plt.legend()

In [ ]:
N = 100
pdfs_normal = np.zeros((N, len(xpdf)))
pdfs_edge = np.zeros_like(pdfs_normal)
for i in tqdm(range(N)):
    pdfs_normal[i] = try_only_normal(i)
    pdf1, pdf2 = try_method(i)
    pdfs_edge[i] = pdf1*(1-py0) + pdf2*py0

In [ ]:
plt.subplots(1, 1, figsize=(12, 10))
plt.plot(xpdf, pdfreal, label="Real")
mu1 = np.mean(pdfs_normal, axis=0)
sigma1 = np.std(pdfs_normal, axis=0)
mse1 = np.mean(np.sqrt((pdfs_normal-pdfreal)**2), axis=0)
plt.plot(xpdf, mu1, label="Mean est1")
plt.fill_between(xpdf, mu1-sigma1, mu1+sigma1, color=(1, .8, .8), label="Uncertainty est1")
mu2 = np.mean(pdfs_edge, axis=0)
sigma2 = np.std(pdfs_edge, axis=0)
mse2 = np.mean(np.sqrt((pdfs_edge-pdfreal)**2), axis=0)
plt.plot(xpdf, mu2, label="Mean est2")
plt.fill_between(xpdf, mu2-sigma2, mu2+sigma2, color=(.8, 1, .8), label="Uncertainty est2")
plt.legend()
plt.ylim(0, 0.3)

In [ ]:
plt.plot(xpdf, mse1)
plt.plot(xpdf, mse2)

In [ ]:
plt.plot(xpdf, mse2/mse1)
plt.ylim(0, 2)

In [ ]:
n1 = 500
n2 = 500
ythreshold = 2
gm = GaussianMixture([[0, 0]], 
                     [[[2, 1.3], [1.3, 1]]])
(xpdf, ypdf), zpdf = gm.pdf(minx=[-5, -5], maxx=[5, 5])
pdfreal = np.sum(zpdf, axis=0) * np.mean(np.diff(ypdf[:, 0]))
plt.contourf(xpdf, ypdf, zpdf)

dx = np.mean(np.diff(xpdf[0, :]))
dy = np.mean(np.diff(ypdf[:, 0]))
iythreshold = np.argmin(np.abs(ypdf[:, 0] - ythreshold))

In [ ]:
def try_method2(seed):
    np.random.seed(seed)

    data1 = gm.generate_samples(n1)
    invpycest = n1/np.sum(data1[:, 1] > ythreshold)
    data2 = gm.generate_samples(int(n2*invpycest))
    pycest = np.mean(data2[:, 1] > ythreshold)
    data2 = data2[data2[:, 1] > ythreshold, :]

    kx1 = KDE(data1, scaling=True)
    kx1.compute_bandwidth()
    pdfestxy1 = kx1.score_samples(np.array([xpdf, ypdf]).T).T
    cdfest1 = np.cumsum(np.trapz(pdfestxy1, axis=0)) * dy * dx
    pdfest1 = np.gradient(cdfest1) / dx

    kx2 = KDE(data2, scaling=True)
    kx2.compute_bandwidth()
    pdfestxy2 = kx2.score_samples(np.array([xpdf, ypdf]).T).T
    normalization = np.trapz(np.sum(pdfestxy2, axis=1)[iythreshold:], ypdf[iythreshold:, 0])*dx
    cdfest2 = (np.cumsum(np.trapz(pdfestxy1[:iythreshold], axis=0)) +
               np.cumsum(np.trapz(pdfestxy2[iythreshold:], axis=0))*pycest/normalization)*dy*dx
    pdfest2 = np.gradient(cdfest2) / dx
    
    return pdfest1, pdfest2

In [ ]:
N = 100
pdfs1 = np.zeros((N, len(xpdf)))
pdfs2 = np.zeros_like(pdfs1)
for i in tqdm(range(N)):
    pdf1, pdf2 = try_method2(i)
    pdfs1[i] = pdf1
    pdfs2[i] = pdf2

In [ ]:
plt.subplots(1, 1, figsize=(12, 10))
plt.plot(xpdf[0], pdfreal, label="Real")
mu1 = np.mean(pdfs1, axis=0)
sigma1 = np.std(pdfs1, axis=0)
plt.plot(xpdf[0], mu1, label="Mean est1")
plt.fill_between(xpdf[0], mu1-sigma1, mu1+sigma1, color=(1, .8, .8), label="Uncertainty est1")
mu2 = np.mean(pdfs2, axis=0)
sigma2 = np.std(pdfs2, axis=0)
plt.plot(xpdf[0], mu2, label="Mean est2")
plt.fill_between(xpdf[0], mu2-sigma2, mu2+sigma2, color=(.8, 1, .8), label="Uncertainty est2")
plt.legend()
plt.ylim(0, 0.3)

In [ ]:
xpdf = np.linspace(-5, 5, 101)
pdfreal = np.exp(-xpdf**2/4) / np.sqrt(4*np.pi)

In [ ]:
def try_method3(seed):
    np.random.seed(seed)
    
    data1 = gm.generate_samples(n1)
    invpycest = n1/np.sum(data1[:, 1] > ythreshold)
    data2 = gm.generate_samples(int(n2*invpycest))
    pycest = np.mean(data2[:, 1] > ythreshold)
    data2 = data2[data2[:, 1] > ythreshold, 0]
    
    pycest = 1-scipy.stats.norm.cdf(ythreshold)
    
    data1_ec = data1[data1[:, 1] > ythreshold, 0]
    data_normal = data1[data1[:, 1] <= ythreshold, 0]
    data_ec = np.concatenate((data1_ec, data2))
    
    kde1 = KDE(data1[:, 0])
    kde1.compute_bandwidth()
    pdf1 = kde1.score_samples(xpdf)
    kde_normal = KDE(data_normal)
    kde_normal.compute_bandwidth()
    pdf_normal = kde_normal.score_samples(xpdf)
    kde_ec = KDE(data_ec)
    kde_ec.set_bandwidth(kde_normal.get_bandwidth())
    pdf_ec = kde_ec.score_samples(xpdf)
    
    return pdf1, pdf_ec*pycest + pdf_normal*(1-pycest)

In [ ]:
N = 100
pdfs1 = np.zeros((N, len(xpdf)))
pdfs2 = np.zeros_like(pdfs1)
for i in tqdm(range(N)):
    pdf1, pdf2 = try_method3(i)
    pdfs1[i] = pdf1
    pdfs2[i] = pdf2

In [ ]:
plt.subplots(1, 1, figsize=(12, 10))
plt.plot(xpdf, pdfreal, label="Real")
mu1 = np.mean(pdfs1, axis=0)
sigma1 = np.std(pdfs1, axis=0)
plt.plot(xpdf, mu1, label="Mean est1")
plt.fill_between(xpdf, mu1-sigma1, mu1+sigma1, color=(1, .8, .8), label="Uncertainty est1")
mu2 = np.mean(pdfs2, axis=0)
sigma2 = np.std(pdfs2, axis=0)
plt.plot(xpdf, mu2, label="Mean est2")
plt.fill_between(xpdf, mu2-sigma2, mu2+sigma2, color=(.8, 1, .8), label="Uncertainty est2")
plt.legend()
plt.ylim(0, 0.3)

In [ ]:
plt.plot(xpdf, pdfreal-mu1, label="Error 1")
plt.plot(xpdf, pdfreal-mu2, label="Error 2")
plt.title("Error with real pdf")
plt.legend()

In [ ]:
plt.plot(xpdf, sigma1, label="Estimate 1")
plt.plot(xpdf, sigma2, label="Estimate 2")
plt.title("Standard deviation estimation")
plt.legend()

In [ ]:
plt.plot(xpdf, sigma2/sigma1)